# Data Cleaning

In [2]:
import pandas as pd

In [3]:
dataframe = pd.read_csv("HistoricalQuotes.csv")

In [4]:
clean_headers = [None]*len(dataframe.columns)

Removing spaces of headers

In [5]:
for column_index in range(0, len(dataframe.columns)):
    clean_headers[column_index] = dataframe.columns[column_index].strip()

Updating headers

In [6]:
dataframe.columns = clean_headers

In [7]:
dataframe.columns

Index(['Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low'], dtype='object')

Removing '$' from columns and changing datatype string to float

In [8]:
dataframe['Close/Last'] = dataframe['Close/Last'].str.replace('$', '').astype(float)
dataframe['Open'] = dataframe['Open'].str.replace('$', '').astype(float)
dataframe['High'] = dataframe['High'].str.replace('$', '').astype(float)
dataframe['Low'] = dataframe['Low'].str.replace('$', '').astype(float)

Saving filtered dataframe

In [9]:
dataframe.to_csv("FilteredHistoricalQuotes.csv", index=False)

# Uploading File to S3 bucket

In [25]:
import boto3 # boto3 library is used to access aws s3

Getting ACCESS_KEY and SECRET_ACCESS_KEY from aws_credentials file

In [6]:
with open('aws_credentials') as f:
    lines = f.read().splitlines()

Connecting to aws S3

In [17]:
s3 = boto3.client(
    's3',
    aws_access_key_id=lines[0],
    aws_secret_access_key=lines[1],
)

Creating bucket

In [20]:
s3.create_bucket(
    Bucket='historicalstockpricesbucket',
    CreateBucketConfiguration= {'LocationConstraint': 'ap-south-1'}
)

{'Location': 'http://historicalstockpricesbucket.s3.amazonaws.com/',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Wed, 13 May 2020 07:33:59 GMT',
   'location': 'http://historicalstockpricesbucket.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'x-amz-id-2': '3wKrVj2MaK7htwIeX/LQPFG1BSb3MiG5kR8yXSxO9TGWOQmbnIFyQBDUohqdi990wqbDzLu4WSU=',
   'x-amz-request-id': '4F205AE8CFD23DEE'},
  'HTTPStatusCode': 200,
  'HostId': '3wKrVj2MaK7htwIeX/LQPFG1BSb3MiG5kR8yXSxO9TGWOQmbnIFyQBDUohqdi990wqbDzLu4WSU=',
  'RequestId': '4F205AE8CFD23DEE',
  'RetryAttempts': 0}}

Listing buckets

In [22]:
response = s3.list_buckets()

for bucket in response['Buckets']:
    print(bucket)

{'Name': 'historicalstockpricesbucket', 'CreationDate': datetime.datetime(2020, 5, 13, 7, 34, tzinfo=tzutc())}


Uploading files

In [24]:
s3.upload_file("FilteredHistoricalQuotes.csv", 'historicalstockpricesbucket', 'GoogleHistoricalQuotes.csv')